In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint
from keras.models import model_from_yaml




2022-02-17 19:38:39.504997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-17 19:38:39.505037: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from keras.models import model_from_json

# Build our network from loaded architecture and weights
with open('model.json') as model_file:
    architecture = model_file.read()

model = model_from_json(architecture)
model.load_weights('weights-05.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


2022-02-17 19:38:41.098581: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-17 19:38:41.098613: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-17 19:38:41.098636: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (archlinux): /proc/driver/nvidia/version does not exist
2022-02-17 19:38:41.098843: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 20, 128)           86016     
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 39)                5031      
                                                                 
 activation (Activation)     (None, 39)                0         
                                                                 
Total params: 222,631
Trainable params: 222,631
Non-trainable params: 0
_________________________________________________________________


In [4]:
with open("hazes.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()
print("Loaded a corpus of {0} characters".format(len(corpus)))

Loaded a corpus of 41701 characters


In [5]:
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}

corpus_length = len(corpus)
chars = sorted(list(set(corpus)))
sentence_length = 20
num_chars = len(chars)

In [6]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)


def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
        #print(X.shape)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoding[character]] = 1
        
        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            prediction = sample(model.predict(X, verbose=0)[0],0.3)
            generated_text += decoding[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=np.bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = corpus[seed:seed + sentence_length]

        return pattern

In [7]:

seed = make_seed('ik denk daarbij stilletjes aan taart')
print(seed)
txt =  generate(seed)
print(txt)

ik denk daarbij stil


/tmp/ipykernel_112678/2502783996.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
/tmp/ipykernel_112678/2502783996.py:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  activations = np.zeros((1, 1, num_chars), dtype=np.bool)


de ik de daar mij niet noe taar je heben

ik heb ik de het niet verlaan

ik heb is het nu niet daar de maar je weer ik de noog met je weer ik zijn

ik hee in die nooit
ik de krand niet me niet me nooit me kenden

ik heb ik de blijf el voor jou laat je nu niet te dat de dan me kenderen dan heer
geen dan ik de geen straan

het ik de dan met de blijd
tor dat ik de krand

ik je het niet mijn niet noriet me meer

dat ik dat ik de houd ik de dan jou niet me niet noor de laat

ik heb ik niet me me niet
